# जनावर विशेषज्ञ प्रणाली कार्यान्वयन गर्दै

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) बाट एक उदाहरण।

यस नमूनामा, हामी केही भौतिक विशेषताहरूमा आधारित जनावर निर्धारण गर्न एक साधारण ज्ञान-आधारित प्रणाली कार्यान्वयन गर्नेछौं। प्रणालीलाई निम्न AND-OR रूखद्वारा प्रतिनिधित्व गर्न सकिन्छ (यो सम्पूर्ण रूखको एक भाग हो, हामी सजिलै केहि थप नियमहरू थप्न सक्छौं):

![](../../../../../../translated_images/ne/AND-OR-Tree.5592d2c70187f283.webp)


## हाम्रो आफ्नै विशेषज्ञ प्रणाली शेल पछाडि अन्वेषणसहित

उत्पादन नियमहरूमा आधारित ज्ञान प्रतिनिधित्वको लागि सरल भाषा परिभाषित गर्ने प्रयास गरौं। हामी नियमहरू परिभाषित गर्न कुञ्जीशब्दहरू रूपमा Python कक्षाहरू प्रयोग गर्नेछौं। आधारभूत रूपमा ३ प्रकारका कक्षाहरू हुनेछन्ः
* `Ask` प्रयोगकर्तालाई सोध्नुपर्ने प्रश्न प्रतिनिधित्व गर्छ। यसले सम्भावित उत्तरहरूको सेट समावेश गर्छ।
* `If` नियम प्रतिनिधित्व गर्छ, र यो नियमको सामग्री राख्नको लागि केवल एक व्याकरणिक सजावट हो
* `AND`/`OR` रूखका AND/OR शाखाहरूलाई प्रतिनिधित्व गर्नका लागि कक्षाहरू हुन्। यी भित्रका तर्कहरूको सूची मात्र राख्छन्। कोडलाई सरल बनाउन सबै कार्यक्षमताहरू अभिभावक कक्षा `Content` मा परिभाषित छन्।


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

हाम्रो प्रणालीमा, कार्य स्मृति मा **तथ्यहरू** को सूची **गुण-अर्थ जोडीहरूको रूपमा** समावेश हुन्छ। ज्ञानभण्डारलाई एउटा ठूलो शब्दकोशको रूपमा परिभाषित गर्न सकिन्छ जसले क्रियाकलापहरू (नयाँ तथ्यहरू जुन कार्य स्मृतिमा थपिनुपर्छ) लाई सशर्तहरूमा नक्साङ्कन गर्छ, जुन AND-OR अभिव्यक्तिहरूको रूपमा व्यक्त गरिएको हुन्छ। साथै, केही तथ्यहरूलाई `पूछ्न` सकिन्छ।


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

पछाडिको निरुपण गर्नका लागि, हामी `Knowledgebase` कक्षा परिभाषित गर्नेछौं। यसले समावेश गर्नेछ:
* कार्यरत `memory` - एउटा शब्दकोश जसले विशेषताहरूलाई मानहरूमा नक्सांकन गर्छ
* माथि परिभाषित स्वरूपमा Knowledgebase `rules`

दुई मुख्य विधिहरू हुन्:
* `get` विशेषताको मान प्राप्त गर्न, आवश्यक परेमा निरुपण गर्दै। उदाहरणका लागि, `get('color')` ले रंग स्लटको मान प्राप्त गर्नेछ (आवश्यक परेमा सोध्नेछ, र पछि प्रयोगको लागि मान कार्यरत स्मृतिमा भण्डारण गर्नेछ)। यदि हामी `get('color:blue')` सोध्यौं भने, यो रंग सोध्नेछ, र त्यसपछि रंग अनुसार `y`/`n` मान फर्काउनेछ।
* `eval` वास्तविक निरुपण सञ्चालन गर्दछ, अर्थात AND/OR रुखमा यात्रा गर्ने, उप-लक्ष्यहरू मूल्याङ्कन गर्ने आदि।


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

अब हामी हाम्रो जनावर ज्ञान आधार परिभाषित गर्छौं र परामर्श सम्पन्न गर्छौं। ध्यान दिनुहोस् कि यो कलले तपाईंलाई प्रश्न सोध्नेछ। तपाईंले हो-होइन प्रश्नहरूको लागि `y`/`n` टाइप गरेर उत्तर दिन सक्नुहुन्छ, वा लामो बहुविकल्पी उत्तरहरू भएका प्रश्नहरूको लागि संख्या (0..N) निर्धारण गरेर उत्तर दिन सक्नुहुन्छ।


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## अग्रगामी अनुमानका लागि Experta को प्रयोग

अर्को उदाहरणमा, हामी ज्ञान प्रतिनिधित्वका लागि उपलब्ध पुस्तकालयहरूमध्ये एक, [Experta](https://github.com/nilp0inter/experta) को प्रयोग गरेर अग्रगामी अनुमान कार्यान्वयन गर्ने प्रयास गर्नेछौं। **Experta** पायथनमा अग्रगामी अनुमान प्रणालीहरू सिर्जना गर्नको लागि एक पुस्तकालय हो, जसलाई पुरानो कक्षा प्रणाली [CLIPS](http://www.clipsrules.net/index.html) सँग समान डिजाइन गरिएको छ।

हामीले अगाडि श्रृंखला पनि सजिलै आफैं कार्यान्वयन गर्न सक्थ्यौं, तर साधारण कार्यान्वयनहरू प्रायः धेरै कुशल हुँदैनन्। नियम मिलानको प्रभावकारीतालाई बढाउनका लागि विशेष एल्गोरिदम [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) प्रयोग गरिन्छ।


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

हामी हाम्रो प्रणालीलाई `KnowledgeEngine` को सबक्लास हुने क्लासको रूपमा परिभाषित गर्नेछौं। प्रत्येक नियमलाई `@Rule` एनोटेशन भएको अलग फंक्शनद्वारा परिभाषित गरिन्छ, जसले नियम कहिले लागू हुने हो भनी निर्दिष्ट गर्छ। नियमभित्र, हामी `declare` फंक्शनको प्रयोग गरेर नयाँ तथ्यहरू थप्न सक्छौं, र ती तथ्यहरू थप्दा अग्रगामी अनुमान इन्जिनद्वारा केही अरु नियमहरू लागू हुनेछन्।


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

एक पटक हामीले ज्ञानको आधार परिभाषित गरेपछि, हामीले हाम्रो कार्य सम्झनामा केही प्रारम्भिक तथ्यहरू राख्छौं, र त्यसपछि inference प्रदर्शन गर्न `run()` मेथड कल गर्छौं। तपाईंले नतिजाको रूपमा देख्न सक्नुहुन्छ कि नयाँ अनुमानित तथ्यहरू कार्य सम्झनामा थपिन्छन्, जसमा जनावरबारे अन्तिम तथ्य पनि समावेश छ (यदि हामीले सबै प्रारम्भिक तथ्यहरू ठीकसँग सेट अप गरेका छौं भने)।


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:
यो दस्तावेज AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) को प्रयोग गरेर अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया बुझ्नुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज यसको मातृ भाषामा अधिकृत स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि भ्रम वा गलत व्याख्याहरूको लागि हामी उत्तरदायी छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
